In [ ]:
%pip install neuralprophet[live]
%pip install numpy

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from neuralprophet import NeuralProphet
#https://github.com/ourownstory/neural_prophet

In [29]:
#Read data
csv_file = 'CPIUS.csv'
df = pd.read_csv('DataSet\\'+csv_file)
df.dropna(inplace=True)

df_clean = df.loc[:,['DATE','CPILFENS']]
df_clean.rename(columns={'DATE':'ds','CPILFENS':'y'},inplace=True)
df_clean['ds'] = pd.to_datetime(df_clean.ds)
#Subseting data, Leave some data for forcecasting
df_train = df_clean[:-5].copy()
df_train.dropna(inplace=True)
df_clean.dropna(inplace=True)

In [33]:
neuralprop = NeuralProphet(
    drop_missing=True,
    growth="linear",
    trend_reg=0,
    trend_reg_threshold=False,
    yearly_seasonality='auto',
    weekly_seasonality='auto',
    daily_seasonality='auto',
    seasonality_mode='additive',
    seasonality_reg=0,
    n_lags=7,
    #NN stuff
    batch_size=32,
    num_hidden_layers=2,
    d_hidden=4,
    learning_rate=0.001,
    epochs=100,
    loss_func='Huber',
    optimizer= "AdamW",
    normalize='auto',
    impute_missing=True
    
)

In [34]:
metrics = neuralprop.fit(df_train, freq='D')
metrics.head()

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 91.465% of the data.
WARNING - (NP.df_utils._infer_frequency) - Defined frequency D is different than major frequency MS
INFO - (NP.forecaster.__handle_missing_data) - 23077 missing dates added.
WARNING - (NP.forecaster.__handle_missing_data) - 23077 missing values in column y were detected in total. 
INFO - (NP.forecaster.__handle_missing_data) - 23071 NaN values in column y were auto-imputed.
WARNING - (NP.forecaster.__handle_missing_data) - More than 30 consecutive missing values encountered in column y. 6 NA remain after auto-imputation. 
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
WARNING - (NP.time_dataset.drop_nan_after_init) - 20 samples with missing values were dropped from the data. 
Epoch[100/100]: 

,SmoothL1Loss,MAE,RMSE,Loss,RegLoss
0,0.497903,208.815280,251.572168,0.375054,0.0
1,0.485070,205.463555,247.526165,0.365188,0.0
2,0.464812,200.123471,240.987898,0.349764,0.0
3,0.432516,191.499753,230.391317,0.325230,0.0
4,0.384504,178.443374,214.494127,0.289036,0.0


In [137]:
def sacf(array, lag):
    
    mu = np.mean(array)
    numera = 0
    denomina = 0
    for j in range (0,len(array)):
        denomina = denomina + np.power((array[j]-mu),2)
    print(mu, denomina)

    if (lag <= 1):
        for i in range (lag, len(array)):
            print(f'({array[i]}-mu) * ({array[i-lag]}-mu)')
            numera = numera + (array[i]-mu)*(array[i-lag]-mu)
        print(numera/denomina)

    if (lag > 1):
        for i in range (lag, len(array)):
            print(f'({array[i]}-mu) * ({array[i-lag]}-mu)')
            numera = numera + (array[i]-mu)*(array[i-lag]-mu)
        print(numera/denomina)
def ci(array, n):
    result = []
    base = 1.96/np.sqrt(n)
    result.append(base)
    pres = []
    pres.append(1)
    for i in range(0,len(array)-1):
        pres.append(2*np.power(array[i],2))
        result.append(base*np.sqrt(np.sum(pres)))
    print(result)
    